In [1]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import json

In [2]:
def load_embeddings(file_path):
    """Load your existing embeddings function"""
    with open(file_path, 'r') as f:
        data = json.load(f)
    embeddings = data['embeddings']
    names = data['names']
    return embeddings, names

embeddings, names = load_embeddings("saved_embeddings\mtg_embeddings_20250728_153743.json")

In [3]:
# Apply KNN for recommendations
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=6)  # Include the card itself + 5 recommendations
knn.fit(embeddings)

# Example: Get recommendations for a specific card
def get_recommendations(card_name, all_names, all_embeddings, knn, n=10):
    if card_name not in all_names:
        print(f"Card '{card_name}' not found in dataset.")
        return
        
    idx = all_names.index(card_name)
    distances, indices = knn.kneighbors([all_embeddings[idx]], n_neighbors=n+1)
    
    print(f"Recommendations for '{card_name}':")
    for i, idx in enumerate(indices[0][1:]):  # Skip the first one (the card itself)
        print(f"{i+1}. {all_names[idx]}")


In [4]:
get_recommendations("Rocco, Street Chef", names, embeddings, knn, n=10)

Recommendations for 'Rocco, Street Chef':
1. Visions of Phyrexia
2. Rashmi and Ragavan
3. Glimpse the Impossible
4. Faramir, Prince of Ithilien
5. Plargg and Nassari
6. Theater of Horrors
7. Lorehold Excavation
8. Doom Foretold
9. Sliver Gravemother
10. Journey to the Lost City


In [5]:
get_recommendations("Beetleback Chief", names, embeddings, knn, n=10)

Recommendations for 'Beetleback Chief':
1. Goblin Gang Leader
2. Symbiotic Elf
3. Chimney Rabble
4. Head of the Homestead
5. Goblin Instigator
6. Dragon Fodder
7. Krenko's Command
8. Weirding Shaman
9. Elderleaf Mentor
10. Ambassador Oak


In [6]:
get_recommendations("Opt", names, embeddings, knn, n=10)

Recommendations for 'Opt':
1. Consider
2. Watchful Automaton
3. Sigiled Starfish
4. Tidepool Turtle
5. Rummaging Wizard
6. Elven Farsight
7. Geist of the Archives
8. Think Tank
9. Dissolve
10. Glimpse the Sun God
